Milestone-3 is all about "Clinical Note Generation", "automate routine documentation" and "coding tasks using GenAI" .

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


Now i will upload the 3 different datasets i.e clinical data , patient records and lab reports for merging and further operations .

In [2]:
from google.colab import files
uploaded = files.upload()


Saving patient records_heart attack_prediction dataset.xlsx to patient records_heart attack_prediction dataset.xlsx


In [3]:
from google.colab import files
uploaded = files.upload()


Saving clinical_data_heartfailure.csv to clinical_data_heartfailure.csv


In [4]:
from google.colab import files
uploaded = files.upload()


Saving lab_reports_datset.xlsx to lab_reports_datset.xlsx


As i have uploaded the 3 datasets here , now i will write code to merge them and display the first few rows of the merged dataset.

In [5]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
try:
    lab_reports_df = pd.read_excel('lab_reports_datset.xlsx')
    clinical_data_df = pd.read_csv('clinical_data_heartfailure.csv')
    patient_records_df = pd.read_excel('patient records_heart attack_prediction dataset.xlsx')
    common_cols = ['Patient ID', 'icustay_id']
    # Merging the dataframes.
    merged_df = pd.merge(lab_reports_df, clinical_data_df, on=common_cols, how='outer')
    merged_df = pd.merge(merged_df, patient_records_df, on=common_cols, how='outer')
    # Displaying the first few rows of the merged dataframe.
    display(merged_df.head())
except FileNotFoundError as e:
    print(f"Error loading file: {e}. Please ensure the files were uploaded correctly.")
except KeyError as e:
    print(f"Error merging dataframes: {e}. Please ensure the common columns exist in all files.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


,Patient ID,subject_id,Sex_x,icustay_id,heart_rate,Blood Pressure_x,platelets_x,BMI_x,oxygen_saturation,respiratory_rate,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
0,AAX1328,51550,Male,296950,97,163/61,228000.00,19.407704,97,5,...,1.920825,115859,19.407704,485,7,8,Vietnam,Asia,Northern Hemisphere,1
1,ACQ6112,96888,Male,274028,91,149/100,210000.00,19.947050,97,16,...,2.615811,162055,19.947050,554,6,7,United Kingdom,Europe,Northern Hemisphere,0
2,ADG8069,96091,Male,207533,96,160/70,263358.03,33.085489,95,16,...,10.367514,105818,33.085489,569,0,6,Japan,Asia,Northern Hemisphere,0
3,AGH6728,51202,Female,292084,99,145/82,242000.00,36.873611,98,16,...,4.213498,59008,36.873611,788,7,10,Colombia,South America,Northern Hemisphere,0
4,AIW4911,93610,Male,250740,96,123/95,362000.00,19.497133,95,33,...,6.956697,50675,19.497133,63,1,6,Colombia,South America,Northern Hemisphere,1


Now as i have merged the datasets successfully, in my next step, i will generate a new code cell that will take a patient ID as input and then display all the information available for that patient from the merged DataFrame .

In [6]:
patient_id_to_search = input("Enter the Patient ID you want to search for: ")
patient_details_df = merged_df[merged_df['Patient ID'] == patient_id_to_search]
# Checking if any details were found for the entered Patient ID.
if not patient_details_df.empty:
    print(f"Details for Patient ID: {patient_id_to_search}")
    display(patient_details_df)
else:
    print(f"No details found for Patient ID: {patient_id_to_search}")

Enter the Patient ID you want to search for: ADG8069
Details for Patient ID: ADG8069


,Patient ID,subject_id,Sex_x,icustay_id,heart_rate,Blood Pressure_x,platelets_x,BMI_x,oxygen_saturation,respiratory_rate,...,Sedentary Hours Per Day,Income,BMI,Triglycerides,Physical Activity Days Per Week,Sleep Hours Per Day,Country,Continent,Hemisphere,Heart Attack Risk
2,ADG8069,96091,Male,207533,96,160/70,263358.03,33.085489,95,16,...,10.367514,105818,33.085489,569,0,6,Japan,Asia,Northern Hemisphere,0


Now i will access a pre-trained model from "Hugging Face" using the transformers library and the pipeline function.

In [7]:
!pip install transformers
# Importing the pipeline function.
from transformers import pipeline
# Defining the task and the model name from Hugging Face.
task = "summarization"
model_name = "AbhinavSK/cikista-medical-report-summarizer" # Pre-trained Model name.
# Loading the pipeline.
try:
    nlp_pipeline = pipeline(task, model=model_name)
    print(f"Successfully loaded pipeline for task '{task}' using model '{model_name}'")
except Exception as e:
    print(f"Error loading pipeline: {e}")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


adapter_config.json:   0%|          | 0.00/858 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Error loading pipeline: No package metadata was found for bitsandbytes


Now i will summarize the patient details using the loaded model. For that i need to extract the relevant information from the "patient_details_df" DataFrame and format it as text that the summarization model can process.Then i can use the nlp_pipeline to generate the summary.

In [8]:
patient_text = ""
for col in patient_details_df.columns:
    patient_text += f"{col}: {patient_details_df[col].iloc[0]}\n"
print("Patient Details Text for Summarization:")
print(patient_text)
# Using the loaded summarization pipeline to summarize the text.
try:
    summary = nlp_pipeline(patient_text, max_length=150, min_length=30, do_sample=False)
    print("\nSummary of Patient Details:")
    print(summary[0]['summary_text'])
except Exception as e:
    print(f"Error during summarization: {e}")

Patient Details Text for Summarization:
Patient ID: ADG8069
subject_id: 96091
Sex_x: Male
icustay_id: 207533
heart_rate: 96
Blood Pressure_x: 160/70
platelets_x: 263358.03
BMI_x: 33.08548935
oxygen_saturation: 95
respiratory_rate: 16
temperature: 37.3
label: 0
Previous Heart Problems_x: 1
Medication Use_x: 0
Stress Level_x: 9
Age_x: 39
anaemia: 0
creatinine_phosphokinase: 582
Diabetes_x: 1
ejection_fraction: 50
high_blood_pressure: 0
platelets_y: 263358.03
serum_creatinine: 1.18
serum_sodium: 137
Sex_y: Male
Cholesterol_x: 255
Blood Pressure_y: 160/70
DEATH_EVENT: 0
BMI_y: 33.08548935
Diet_x: Unhealthy
Previous Heart Problems_y: 1
Alcohol Consumption_x: 1
Previous Heart Problems.1: 1
Medication Use_y: 0
Stress Level_y: 9
Age_y: 39
Sex: Male
Cholesterol_y: 255
Blood Pressure: 160/70
platelets: 263358.03
Heart Rate: 81
Diabetes_y: 1
Family History: 0
Smoking: 1
Obesity: 1
Alcohol Consumption_y: 1
Exercise Hours Per Week: 14.1439683395214
Diet_y: Unhealthy
Previous Heart Problems: 1
Medic

Now i will create a list to store the text representation of each patient's details.

In [9]:
all_patients_text = []
# Iterating through each row of the merged_df DataFrame.
for index, row in merged_df.iterrows():
    patient_text = ""
    for col in merged_df.columns:
        patient_text += f"{col}: {row[col]}\n"
    all_patients_text.append(patient_text)
# printing the first few entries to see the format.
print("First 5 entries of the prepared dataset:")
for i in range(min(5, len(all_patients_text))):
    print(f"--- Patient {i+1} ---")
    print(all_patients_text[i])

First 5 entries of the prepared dataset:
--- Patient 1 ---
Patient ID: AAX1328
subject_id: 51550
Sex_x: Male
icustay_id: 296950
heart_rate: 97
Blood Pressure_x: 163/61
platelets_x: 228000.0
BMI_x: 19.40770421
oxygen_saturation: 97
respiratory_rate: 5
temperature: 36.2
label: 1
Previous Heart Problems_x: 1
Medication Use_x: 1
Stress Level_x: 3
Age_x: 28
anaemia: 0
creatinine_phosphokinase: 60
Diabetes_x: 1
ejection_fraction: 35
high_blood_pressure: 0
platelets_y: 228000.0
serum_creatinine: 1.2
serum_sodium: 135
Sex_y: Male
Cholesterol_x: 237
Blood Pressure_y: 163/61
DEATH_EVENT: 0
BMI_y: 19.40770421
Diet_x: Unhealthy
Previous Heart Problems_y: 1
Alcohol Consumption_x: 0
Previous Heart Problems.1: 1
Medication Use_y: 1
Stress Level_y: 3
Age_y: 28
Sex: Male
Cholesterol_y: 237
Blood Pressure: 163/61
platelets: 228000.0
Heart Rate: 73
Diabetes_y: 1
Family History: 0
Smoking: 1
Obesity: 1
Alcohol Consumption_y: 0
Exercise Hours Per Week: 13.5788451470325
Diet_y: Unhealthy
Previous Heart Prob

 Now i will access a preprocessed diagnosis model from "Hugging Face" ,and then in my next step i will use this model with our merged_df dataset.

In [10]:
!pip install transformers
# Importing the pipeline function.
from transformers import pipeline
# Defining the task and the model name from Hugging Face.
task = "diagnosis"
model_name = "NeoAivara/Medical-Diagnosis-COT-DeepSeek" # Pre-processed model name.
# Loading the pipeline.
try:
    nlp_pipeline = pipeline(task, model=model_name)
    print(f"Successfully loaded pipeline for task '{task}' using model '{model_name}'")
except Exception as e:
    print(f"Error loading pipeline: {e}")

config.json: 0.00B [00:00, ?B/s]

adapter_config.json: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Error loading pipeline: "Unknown task diagnosis, available tasks are ['audio-classification', 'automatic-speech-recognition', 'depth-estimation', 'document-question-answering', 'feature-extraction', 'fill-mask', 'image-classification', 'image-feature-extraction', 'image-segmentation', 'image-text-to-text', 'image-to-image', 'image-to-text', 'keypoint-matching', 'mask-generation', 'ner', 'object-detection', 'question-answering', 'sentiment-analysis', 'summarization', 'table-question-answering', 'text-classification', 'text-generation', 'text-to-audio', 'text-to-speech', 'text2text-generation', 'token-classification', 'translation', 'video-classification', 'visual-question-answering', 'vqa', 'zero-shot-audio-classification', 'zero-shot-classification', 'zero-shot-image-classification', 'zero-shot-object-detection', 'translation_XX_to_YY']"


In [12]:
!pip install transformers
# Importing the pipeline function.
from transformers import pipeline
# Defining the model name.
model_name = "NeoAivara/Medical-Diagnosis-COT-DeepSeek"
# Attemptting to load the pipeline with a general text generation task.
try:
    nlp_pipeline_diagnosis = pipeline("text2text-generation", model=model_name)
    print(f"Successfully loaded pipeline for model '{model_name}' with 'text2text-generation' task.")
except Exception as e:
    print(f"Error loading pipeline with 'text2text-generation': {e}")
    try:
        nlp_pipeline_diagnosis = pipeline("text-generation", model=model_name)
        print(f"Successfully loaded pipeline for model '{model_name}' with 'text-generation' task.")
    except Exception as e_gen:
         print(f"Error loading pipeline with 'text-generation': {e_gen}")
         print("Could not load the model with a standard text generation task. Please check the model card on Hugging Face for correct usage.")

Error loading pipeline with 'text2text-generation': No package metadata was found for bitsandbytes
Error loading pipeline with 'text-generation': No package metadata was found for bitsandbytes
Could not load the model with a standard text generation task. Please check the model card on Hugging Face for correct usage.
